# Lesson 3: Agentic Search

In [1]:
# libraries
from dotenv import load_dotenv
import os
from tavily import TavilyClient

# load environment variables from .env file
_ = load_dotenv()

# connect
client = TavilyClient(api_key=os.environ.get("TAVILY_API_KEY"))

In [2]:
# run search
result = client.search("What is in Nvidia's new Blackwell GPU?",
                       include_answer=True)

# print the answer
result["answer"]


"The new Nvidia Blackwell GPU features the Blackwell B200 tensor core chip, which is claimed to be the company's most powerful single-chip GPU with 208 billion transistors. Nvidia states that this chip can reduce AI inference operating costs and energy consumption by up to 25 times compared to its predecessor, the H100. The Blackwell architecture is designed to power the RTX 50-series graphics cards and is expected to drive the next generation of AI supercomputers, potentially offering more than quadruple the performance of previous models."

## Regular search

In [3]:
# choose location (try to change to your own city!)

city = "San Francisco"

query = f"""
    what is the current weather in {city}?
    Should I travel there today?
    "weather.com"
"""

> Note: search was modified to return expected results in the event of an exception. High volumes of student traffic sometimes cause rate limit exceptions.

In [4]:
import requests
from bs4 import BeautifulSoup
from duckduckgo_search import DDGS
import re

ddg = DDGS()

def search(query, max_results=6):
    try:
        results = ddg.text(query, max_results=max_results)
        return [i["href"] for i in results]
    except Exception as e:
        print(f"returning previous results due to exception reaching ddg.")
        results = [ # cover case where DDG rate limits due to high deeplearning.ai volume
            "https://weather.com/weather/today/l/USCA0987:1:US",
            "https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8",
        ]
        return results  


for i in search(query):
    print(i)

https://weather.com/weather/hourbyhour/l/54f9d8baac32496f6b5497b4bf7a277c3e2e6cc5625de69680e6169e7e38e9a8
https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US
https://weather.com/weather/tenday/l/San+Francisco+CA?canonicalCityId=45cf83277ba620e7dc8a0fe8b6eda89925a3e6d2e1bdfef3f74a1590017bd70d
https://weather.com/weather/today/l/San+Diego+CA?canonicalCityId=cb5c473781cc06501376639dce8f0823a99187dcb42c79471a4303c076d66452
https://weather.com/weather/tenday/l/Santa+Clara+CA?canonicalCityId=bf2c6bcebb55ec0cfc160ec23a861ed6245ecd32baa02552b65257ba022c3c8d


In [5]:
def scrape_weather_info(url):
    """Scrape content from the given URL"""
    if not url:
        return "Weather information could not be found."
    
    # fetch data
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return "Failed to retrieve the webpage."

    # parse result
    soup = BeautifulSoup(response.text, 'html.parser')
    return soup


> Note: This produces a long output, you may want to right click and clear the cell output after you look at it briefly to avoid scrolling past it.

In [6]:
# use DuckDuckGo to find websites and take the first result
url = search(query)[0]

# scrape first wesbsite
soup = scrape_weather_info(url)

print(f"Website: {url}\n\n")
print(str(soup.body)[:50000]) # limit long outputs

Website: https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US


<body><div class="appWrapper DaybreakLargeScreen LargeScreen lightTheme twcTheme DaybreakLargeScreen--appWrapper--3kKUE gradients--clearNight--2-RGV gradients--clearNight-top--3tFbG" id="appWrapper"><div class="region-meta"><div class="removeIfEmpty" id="WxuHtmlHead-meta-"></div><div class="removeIfEmpty" id="WxuNewsroom-meta-bc9f40d5-d941-4fd8-bae2-2d8d63a38bb3"></div></div><div class="region-topAds regionTopAds DaybreakLargeScreen--regionTopAds--2kcLJ"><div class="removeIfEmpty" id="WxuAd-topAds-53dce052-5465-4609-a555-c3a20ab64ab0"><div class="adWrapper BaseAd--adWrapper--1dWzA BaseAd--card--2G19u BaseAd--hide--3LkRr"><div class="adLabel BaseAd--adLabel--1LMu9">Advertisement</div><div class="ad_module BaseAd--ad_module--DeGCt subs-undefined BaseAd--placeholder--3IQan" id="WX_Hidden"></div></div></div><div class="removeIfEmpty" id="WxuAd-topAds-fe926b10-58bc-448a-ab09-47e692334250"><div class="adWrapper Bas

In [7]:
# extract text
weather_data = []
for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
    text = tag.get_text(" ", strip=True)
    weather_data.append(text)

# combine all elements into a single string
weather_data = "\n".join(weather_data)

# remove all spaces from the combined text
weather_data = re.sub(r'\s+', ' ', weather_data)
    
print(f"Website: {url}\n\n")
print(weather_data)

Website: https://weather.com/weather/today/l/San+Francisco+CA+USCA0987:1:US


recents Specialty Forecasts San Francisco, CA Beach Hazard Statement Today's Forecast for San Francisco, CA Morning Afternoon Evening Overnight Weather Today in San Francisco, CA 5:47 am 8:29 pm Don't Miss Hourly Forecast Now 3 am 4 am 5 am 6 am Outside That's Not What Was Expected Daily Forecast Today Fri 07 Sat 08 Sun 09 Mon 10 Radar We Love Our Critters Summer Skin Survival Guide Home, Garage & Garden Editors Pick Weather in your inbox Your local forecast, plus daily trivia, stunning photos and our meteorologists’ top picks. All in one place, every weekday morning. By signing up, you're opting in to receive the Morning Brief email newsletter. To manage your data, visit Data Rights . Terms of Use | Privacy Policy Health News For You Seasonal Spotlight Stay Safe Air Quality Index Air quality is acceptable; however, for some pollutants there may be a moderate health concern for a very small number of people w

## Agentic Search

In [8]:
# run search
result = client.search(query, max_results=1)

# print first result
data = result["results"][0]["content"]

print(data)

{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1717667667, 'localtime': '2024-06-06 2:54'}, 'current': {'last_updated_epoch': 1717667100, 'last_updated': '2024-06-06 02:45', 'temp_c': 9.0, 'temp_f': 48.2, 'is_day': 0, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/night/143.png', 'code': 1030}, 'wind_mph': 2.2, 'wind_kph': 3.6, 'wind_degree': 10, 'wind_dir': 'N', 'pressure_mb': 1011.0, 'pressure_in': 29.86, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 100, 'cloud': 25, 'feelslike_c': 7.1, 'feelslike_f': 44.8, 'windchill_c': 11.0, 'windchill_f': 51.9, 'heatindex_c': 12.2, 'heatindex_f': 54.0, 'dewpoint_c': 9.5, 'dewpoint_f': 49.2, 'vis_km': 8.0, 'vis_miles': 4.0, 'uv': 1.0, 'gust_mph': 12.0, 'gust_kph': 19.4}}


In [9]:
import json
from pygments import highlight, lexers, formatters

# parse JSON
parsed_json = json.loads(data.replace("'", '"'))

# pretty print JSON with syntax highlighting
formatted_json = json.dumps(parsed_json, indent=4)
colorful_json = highlight(formatted_json,
                          lexers.JsonLexer(),
                          formatters.TerminalFormatter())

print(colorful_json)


{
    "location": {
        "name": "San Francisco",
        "region": "California",
        "country": "United States of America",
        "lat": 37.78,
        "lon": -122.42,
        "tz_id": "America/Los_Angeles",
        "localtime_epoch": 1717667667,
        "localtime": "2024-06-06 2:54"
    },
    "current": {
        "last_updated_epoch": 1717667100,
        "last_updated": "2024-06-06 02:45",
        "temp_c": 9.0,
        "temp_f": 48.2,
        "is_day": 0,
        "condition": {
            "text": "Mist",
            "icon": "//cdn.weatherapi.com/weather/64x64/night/143.png",
            "code": 1030
        },
        "wind_mph": 2.2,
        "wind_kph": 3.6,
        "wind_degree": 10,
        "wind_dir": "N",
        "pressure_mb": 1011.0,
        "pressure_in": 29.86,
        "precip_mm": 0.0,
        "precip_in": 0.0,
        "humidity": 100,
        "cloud": 25,
        "feelslike_c": 7.1,
        "feelslike_f": 44.8,
        "windchill_c": 11.0,
        "windchill_f

<img src="./google_sample.png" width="800" height="600">